In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from nlstruct.dataloaders.medic import get_raw_medic

In [3]:
from __future__ import absolute_import
import argparse
import numpy as np
import torch
import os
import sys
import logging
import pdb

sys.path.insert(0,'/home/ytaille/AttentionSegmentation')

from allennlp.data import Vocabulary
from allennlp.data.iterators import DataIterator
# import allennlp.data.dataset_readers as Readers
import AttentionSegmentation.reader as Readers

# import model as Models
import AttentionSegmentation.model.classifiers as Models

from AttentionSegmentation.commons.utils import \
    setup_output_dir, read_from_config_file
from AttentionSegmentation.commons.model_utils import \
    construct_vocab, load_model_from_existing
# from AttentionSegmentation.visualization.visualize_attns import \
#     html_visualizer
import AttentionSegmentation.model.attn2labels as SegmentationModels

"""The main entry point

This is the main entry point for training HAN SOLO models.

Usage::

    ${PYTHONPATH} -m AttentionSegmentation/main
        --config_file ${CONFIG_FILE}

"""
args = type('MyClass', (object,), {'content':{}})()
args.config_file = 'Configs/config_ncbi.json'
args.log = 'INFO'
args.loglevel = 'INFO'
args.seed = 1

# Setup Experiment Directory
config = read_from_config_file(args.config_file)
if args.seed > 0:
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if config.get('trainer', None) is not None and \
       config.get('trainer', None).get('cuda_device', -1) > 0:
        torch.cuda.manual_seed(args.seed)
serial_dir, config = setup_output_dir(config, args.loglevel)
logger = logging.getLogger(__name__)

# Load Training Data
TRAIN_PATH = config.pop("train_data_path")
logger.info("Loading Training Data from {0}".format(TRAIN_PATH))
dataset_reader_params = config.pop("dataset_reader")
reader_type = dataset_reader_params.pop("type", None)
assert reader_type is not None and hasattr(Readers, reader_type),\
    f"Cannot find reader {reader_type}"
reader = getattr(Readers, reader_type).from_params(dataset_reader_params)
instances_train = reader.read(file_path=TRAIN_PATH)
instances_train = instances_train
logger.info("Length of {0}: {1}".format(
    "Training Data", len(instances_train)))

# Load Validation Data
VAL_PATH = config.pop("validation_data_path")
logger.info("Loading Validation Data from {0}".format(VAL_PATH))
instances_val = reader.read(VAL_PATH)
instances_val = instances_val
logger.info("Length of {0}: {1}".format(
    "Validation Data", len(instances_val)))

# Load Test Data
TEST_PATH = config.pop("test_data_path", None)
instances_test = None
if TEST_PATH is not None:
    logger.info("Loading Test Data from {0}".format(TEST_PATH))
    instances_test = reader.read(TEST_PATH)
    instances_test = instances_test
    logger.info("Length of {0}: {1}".format(
        "Testing Data", len(instances_test)))

# # Load Pretrained Existing Model
# load_config = config.pop("load_from", None)

# # Construct Vocabulary
vocab_size = config.pop("max_vocab_size", -1)
logger.info("Constructing Vocab of size: {0}".format(vocab_size))
vocab_size = None if vocab_size == -1 else vocab_size
vocab = Vocabulary.from_instances(instances_train,
                                  max_vocab_size=vocab_size)
vocab_dir = os.path.join(serial_dir, "vocab")
assert os.path.exists(vocab_dir), "Couldn't find the vocab directory"
vocab.save_to_files(vocab_dir)

# if load_config is not None:
#     # modify the vocab from the source model vocab
#     src_vocab_path = load_config.pop("vocab_path", None)
#     if src_vocab_path is not None:
#         vocab = construct_vocab(src_vocab_path, vocab_dir)
#         # Delete the old vocab
#         for file in os.listdir(vocab_dir):
#             os.remove(os.path.join(vocab_dir, file))
#         # save the new vocab
#         vocab.save_to_files(vocab_dir)
logger.info("Saving vocab to {0}".format(vocab_dir))
logger.info("Vocab Construction Done")

# # Construct the data iterators
logger.info("Constructing Data Iterators")
data_iterator = DataIterator.from_params(config.pop("iterator"))
data_iterator.index_with(vocab)

logger.info("Data Iterators Done")

# Create the model
logger.info("Constructing The model")
model_params = config.pop("model")
model_type = model_params.pop("type")
assert model_type is not None and hasattr(Models, model_type),\
    f"Cannot find reader {model_type}"
model = getattr(Models, model_type).from_params(
    vocab=vocab,
    params=model_params,
    label_indexer=reader.get_label_indexer()
)
logger.info("Model Construction done")

# visualize = config.pop("visualize", False)
# visualizer = None
# if visualize:
#     visualizer = html_visualizer(vocab, reader)
segmenter_params = config.pop("segmentation")
segment_class = segmenter_params.pop("type")
segmenter = getattr(SegmentationModels, segment_class).from_params(
    vocab=vocab,
    reader=reader,
    params=segmenter_params
)

# logger.info("Segmenter Done")

# print("##################################\nAYYYYYYYYYYYYYYYYYYYYYYYY\n\n\n\n\n\n\n\n###########################")

# exit()


# if load_config is not None:
#     # Load the weights, as specified by the load_config
#     model_path = load_config.pop("model_path", None)
#     layers = load_config.pop("layers", None)
#     load_config.assert_empty("Load Config")
#     assert model_path is not None,\
#         "You need to specify model path to load from"
#     model = load_model_from_existing(model_path, model, layers)
#     logger.info("Pretrained weights loaded")

# logger.info("Starting the training process")



/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


2021-03-12 17:23:51,007: INFO: train_data_path = /home/ytaille/data/resources/medic/ncbi_conll_ner_train.conll
2021-03-12 17:23:51,011: INFO: Loading Training Data from /home/ytaille/data/resources/medic/ncbi_conll_ner_train.conll
2021-03-12 17:23:51,014: INFO: dataset_reader.type = WeakConll2003DatasetReader
2021-03-12 17:23:51,018: INFO: dataset_reader.token_indexers.bert.type = bert-pretrained
2021-03-12 17:23:51,019: INFO: dataset_reader.token_indexers.bert.pretrained_model = ./Data/embeddings/bert-base-multilingual-cased-vocab.txt
2021-03-12 17:23:51,020: INFO: dataset_reader.token_indexers.bert.use_starting_offsets = True
2021-03-12 17:23:51,022: INFO: dataset_reader.token_indexers.bert.do_lowercase = False
2021-03-12 17:23:51,023: INFO: dataset_reader.token_indexers.bert.never_lowercase = None
2021-03-12 17:23:51,024: INFO: dataset_reader.token_indexers.bert.max_pieces = 512
2021-03-12 17:23:51,026: INFO: loading vocabulary file ./Data/embeddings/bert-base-multilingual-cased-voc

0it [00:00, ?it/s]

2021-03-12 17:23:51,171: INFO: Reading instances from lines in file at: /home/ytaille/data/resources/medic/ncbi_conll_ner_train.conll


1803it [00:01, 1416.99it/s]

...

2021-03-12 17:23:53,268: INFO: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz not found in cache, downloading to /tmp/tmp75s0f13b


100%|██████████| 662804195/662804195 [00:18<00:00, 35400184.83B/s]

2021-03-12 17:24:12,475: INFO: copying /tmp/tmp75s0f13b to cache at /home/ytaille/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9


2021-03-12 17:24:18,807: INFO: creating metadata file for /home/ytaille/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
2021-03-12 17:24:18,811: INFO: removing temp file /tmp/tmp75s0f13b
2021-03-12 17:24:18,859: INFO: loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/ytaille/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
2021-03-12 17:24:18,861: INFO: extracting archive file /home/ytaille/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpn801xqy0
2021-03-12 17:24:24,032: INFO: Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "

/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [4]:
# Necessary to add unknown tag to dictionnary to avoid errors later
data_iterator.vocab.add_token_to_namespace("@@UNKNOWN@@", "chunk_tags")

1141

In [5]:
config = read_from_config_file(args.config_file)


In [6]:
from AttentionSegmentation.trainer import Trainer

from nlstruct.utils import  torch_global as tg

trainer = Trainer.from_params(
    model=model,
    base_dir=serial_dir,
    iterator=data_iterator,
    train_data=instances_train,
    validation_data=instances_val,
    segmenter=segmenter,
    params=config.pop("trainer")
)


2021-03-12 17:24:34,581: INFO: trainer.patience = 10
2021-03-12 17:24:34,582: INFO: trainer.validation_metric = +accuracy
2021-03-12 17:24:34,584: INFO: trainer.num_epochs = 50
2021-03-12 17:24:34,585: INFO: trainer.cuda_device = 0
2021-03-12 17:24:34,586: INFO: trainer.grad_norm = None
2021-03-12 17:24:34,586: INFO: trainer.grad_clipping = None
2021-03-12 17:24:34,588: INFO: trainer.num_serialized_models_to_keep = 1
2021-03-12 17:24:37,055: INFO: trainer.optimizer.type = adam
2021-03-12 17:24:37,056: INFO: trainer.optimizer.parameter_groups = [[['.*bert.*'], ConfigTree([('lr', 2e-07)])], [['.*encoder_word.*', '.*attn.*', '.*logit.*'], ConfigTree([('lr', 0.001)])]]
2021-03-12 17:24:37,057: INFO: Converting Params object to dict; logging of default values will not occur when dictionary parameters are used subsequently.
2021-03-12 17:24:37,058: INFO: CURRENTLY DEFINED PARAMETERS: 
2021-03-12 17:24:37,059: INFO: trainer.optimizer.parameter_groups.list.list.lr = 2e-07
2021-03-12 17:24:37,0

In [7]:
# BIT FOR BOOSTING SURROUNDING ATTENTIONS

# attn = torch.Tensor([[0,1,0,1,0], [0,0,1,0,0]])

# attn_boosted = attn.clone()
# nnz = (attn>0).nonzero().t().chunk(chunks=2,dim=0)

# print(nnz)

# new_nnz = [[], []]

# for nz0, nz1 in zip(nnz[0][0].numpy(), nnz[1][0].numpy()):
#     new_nnz[0].extend([nz0,nz0])
#     new_nnz[1].extend([nz1-1,nz1+1])
    
# new_nnz[0] = torch.Tensor([new_nnz[0]]).long()
# new_nnz[1] = torch.Tensor([new_nnz[1]]).long()
# new_nnz = (new_nnz[0], new_nnz[1])
# print(new_nnz)
# attn_boosted[new_nnz] += 0.1

# attn_boosted

In [8]:

# USE BIO BERT
# TRAIN STEP 1 ONLY ON MEDIC LABELS (+ NCBI MENTIONS)
# PREPROCESS / TRAIN / ATTEINDRE BONS SCORES
# GET MEDIC ALTERNATIVE LABELS DANS NLSTRUCT -> TRADUIRE LABELS NCBI VERS MEDIC

# USE ENTROPY INSTEAD OF CROSS ENTROPY -> not rely on labelled data only (rely on model certainty)

# GROUPS : TYPE SEMANTIQUE À LA MENTION (pas utiliser)

# NGRAMS FOR ENTITIES -> not possible with discontinued entities

# Use "separation token" in phrases ?

# Use a limited number of attention heads (not one per class)

# Use same method as Perceval for trajectories (draw closest ones, reduce list, repeat) -> prédiction itérative

# Maybe remove weakly supervised completely?

# Test with Reinforce only after a few epochs

# Facteur de représentation pour pondérer loss de Perceval ?

# Plusieurs facteurs pour constituer la reward

# Facteur de similarité mention extraite / synonyme plutôt que similarité mention / label ?

# Make sure that every trajectory is different -> draw first then use Perceval

# Métrique finale : Est-ce qu'on arrive à choper les CUI ? -> parce que frontières entités dures à déterminer 

# Use only one class ? -> simpler because all mentions are diseases -> MAKE SURE THAT SEVERAL MENTIONS ARE PREDICTABLE

# maybe problem with reinforce algo comes from hyperparameters?? -> USE OPTIMIZER PARAMETER SPECIFICATION

# change objective: instead of WL use RL metrics -> measure on CUI



In [9]:
# USE PERCEVAL WAY OF PREDICTING:
# Entrée: Embeddings tokens + embeddings labels 
# Pour un CUI prédit: récupérer loss Perceval, comparer avec CUI le plus proche?


In [10]:
class NERNet(torch.nn.Module):
    def __init__(self,
                 n_labels,
                 hidden_dim,
                 dropout,
                 n_tokens=None,
                 token_dim=None,
                 embeddings=None,
                 tag_scheme="bio",
                 metric='linear',
                 metric_fc_kwargs=None,
                 ):
        super().__init__()
        if embeddings is not None:
            self.embeddings = embeddings
            if n_tokens is None or token_dim is None:
                if hasattr(embeddings, 'weight'):
                    n_tokens, token_dim = embeddings.weight.shape
                else:
                    n_tokens, token_dim = embeddings.embeddings.weight.shape
        else:
            self.embeddings = torch.nn.Embedding(n_tokens, token_dim) if n_tokens > 0 else None
        assert token_dim is not None, "Provide token_dim or embeddings"
        assert self.embeddings is not None

        dim = (token_dim if n_tokens > 0 else 0)
        self.dropout = torch.nn.Dropout(dropout)
        if tag_scheme == "bio":
            self.crf = BIODecoder(n_labels)
        elif tag_scheme == "bioul":
            self.crf = BIOULDecoder(n_labels)
        else:
            raise Exception()
        if hidden_dim is None:
            hidden_dim = dim
        self.linear = torch.nn.Linear(dim, hidden_dim)
        self.batch_norm = torch.nn.BatchNorm1d(dim)

        n_tags = self.crf.num_tags
        metric_fc_kwargs = metric_fc_kwargs if metric_fc_kwargs is not None else {}
        if metric == "linear":
            self.metric_fc = torch.nn.Linear(dim, n_tags)
        elif metric == "cosine":
            self.metric_fc = CosineSimilarity(dim, n_tags, rescale=rescale, **metric_fc_kwargs)
        elif metric == "ema_cosine":
            self.metric_fc = EMACosineSimilarity(dim, n_tags, rescale=rescale, **metric_fc_kwargs)
        else:
            raise Exception()
    
    def extended_embeddings(self, tokens, mask, **kwargs):
        # Default case here, size <= 512
        # Small ugly check to see if self.embeddings is Bert-like, then we need to pass a mask
        if hasattr(self.embeddings, 'encoder') or hasattr(self.embeddings, 'transformer'):
            return self.embeddings(tokens, mask, **kwargs)[0]
        else:
            return self.embeddings(tokens)

    def forward(self, tokens, mask, tag_embeds=None, return_embeddings=False):
        # Embed the tokens
        scores = None
        # shape: n_batch * sequence * 768
        embeds = self.extended_embeddings(tokens, mask, custom_embeds=tag_embeds)
        state = embeds.masked_fill(~mask.unsqueeze(-1), 0)
        state = torch.relu(self.linear(self.dropout(state)))# + state
        state = self.batch_norm(state.view(-1, state.shape[-1])).view(state.shape)
        scores = self.metric_fc(state)
        return {
            "scores": scores,
            "embeddings": embeds if return_embeddings else None,
        }

In [11]:
from __future__ import absolute_import
import logging
import os
import shutil
import json
from collections import deque
import time
import re
import datetime
import traceback
import numpy as np
from typing import Dict, Optional, List, Tuple, Union, Iterable, Any, Set
import pdb

import torch
import torch.optim.lr_scheduler
from torch.nn.parallel import replicate, parallel_apply
from torch.nn.parallel.scatter_gather import scatter_kwargs, gather
from tensorboardX import SummaryWriter

from itertools import tee

from allennlp.common import Params
from allennlp.common.checks import ConfigurationError
from allennlp.common.util import peak_memory_mb, gpu_memory_mb
from allennlp.common.tqdm import Tqdm
from allennlp.data.instance import Instance
from allennlp.data.iterators.data_iterator import DataIterator
from allennlp.models.model import Model
from allennlp.nn import util
from allennlp.training.learning_rate_schedulers import LearningRateScheduler
from allennlp.training.optimizers import Optimizer

from AttentionSegmentation.commons.trainer_utils import is_sparse,\
    sparse_clip_norm, move_optimizer_to_cuda, TensorboardWriter
# from AttentionSegmentation.visualization.visualize_attns \
#     import html_visualizer
from AttentionSegmentation.model.attn2labels import BasePredictionClass
logger = logging.getLogger(__name__)

TQDM_COLUMNS = 200

import sys
sys.path.insert(0,'/home/ytaille/deep_multilingual_normalization')
from create_classifiers import create_classifiers
from nlstruct.dataloaders import load_from_brat

logger2 = logging.getLogger("nlstruct")
logger2.setLevel(logging.ERROR)

from notebook_utils import *

def _train_epoch(self, epoch: int) -> Dict[str, float]:
        """
        Trains one epoch and returns metrics.
        """
        logger.info(f"Peak CPU memory usage MB: {peak_memory_mb()}")
        if torch.cuda.is_available():
            for gpu, memory in gpu_memory_mb().items():
                logger.info(f"GPU {gpu} memory usage MB: {memory}")

        train_loss = 0.0

        from allennlp.data.fields.array_field import ArrayField

        for i, td in enumerate(self._train_data):
            td.fields['sample_id'] = ArrayField(np.array([i]))

        # Get tqdm for the training batches
        train_generator = self._iterator(self._train_data,
                                         num_epochs=1,
                                         cuda_device=self._iterator_device,
                                         shuffle=True,
                                         )

        train_generator, cp_generator, id_generator = tee(train_generator, 3)

        ids = []

        for ig in id_generator:
            ids.extend([int(sid.item()) for sid in ig['sample_id']])

        shuffled_train_data = [self._train_data[i] for i in ids]

#         train_predictions = self._segmenter.get_predictions(
#                     instances=shuffled_train_data,
#                     iterator = cp_generator,
#                     model=self._model,
#                     cuda_device=self._iterator_device,
#                     verbose=True)


        num_training_batches = self._iterator.get_num_batches(self._train_data)
        train_generator_tqdm = Tqdm.tqdm(train_generator,
                                         total=num_training_batches
                                         )
        self._last_log = time.time()
        last_save_time = time.time()

        batches_this_epoch = 0
        if self._batch_num_total is None:
            self._batch_num_total = 0

        cpt_batch = 0

        # Set the model to "train" mode.
        self._model.train()

        for batch in train_generator_tqdm:
            
            batches_this_epoch += 1
            self._batch_num_total += 1
            batch_num_total = self._batch_num_total
            batch_len = len(batch['labels'])

            # FOR train_predictions:
            # pred/gold is sentence level
            # pred_labels/gold_labels is word level


            # FOR batch:
            # labels is sentence level
            # tags is word level

            # print(train_texts)
            # print("SENTENCE LEVEL")
            # print([tp['gold'] for tp in train_predictions[:10]])
            # print(batch['labels'][:10])

            # print("WORD LEVEL")
            # print([tp['gold_labels'] for tp in train_predictions[:2]])
            # print(batch['tags'][:2])

            # exit()
            
            if epoch <= -1:
                trajectory_scores =  [0]
            else:
                output_dict = self._model(**batch)
#                 attns_single = output_dict['attentions']
                attns = output_dict['attentions']
                
                # Policy is "attention mask": attention scores should be higher if we want to predict CUI
                # Only take words with attention above threshold when predicting with deep norm -> see if it's enough (reward indicates that)
                # REINFORCE algo: (also known as Monte Carlo PG)
                # - draw N trajectories (N attention paths?) -> discretise attentions to make them 1 / 0? -> see if it works with bernoulli first
                # - evaluate each trajectory then sum (maybe add baseline -> subtract mean of all trajectories rewards)
                # - Expected return is given by sum(prob(Ti | W) * reward(Ti)) -> see again if it works with bernoulli first
                # W are WeakL weights 
                # - Gradient ascent of return / gradient descent of negative return

                # Set horizon ? -> number / proportion of attention at 1 per batch
                # Set number of trajectories ? -> maybe make trajectories number vary based on sentence length
                # gamma = 0.9 ? -> used to simulate temporal importance of reward (multiply each step by a certain power of gamma, furthest rewards are less impactful) -> may not be possible to model here
                
                real_tokens = [np.array(b.fields['tokens'].tokens) for b in shuffled_train_data[cpt_batch:cpt_batch+batch_len]]
#                     gold_labels = [np.array(b.fields['tags'].labels) for b in shuffled_train_data[cpt_batch:cpt_batch+batch_len]]
                gold_norm_labels = [np.array(b.fields['chunk_tags'].labels) for b in shuffled_train_data[cpt_batch:cpt_batch+batch_len]]

                save_to_ann(real_tokens, gold_norm_labels, '/home/ytaille/data/tmp/ws_inputs/')

                # NLSTRUCT PART

                bert_name = "bert-base-multilingual-uncased"

                dataset = load_from_brat("/home/ytaille/data/tmp/ws_inputs/")

                if len(dataset['mentions']) == 0:
                    continue

                dataset['mentions']['mention_id'] = dataset['mentions']['doc_id'] +'.'+ dataset['mentions']['mention_id'].astype(str)

                batcher, vocs, mention_ids = preprocess_train(
                    dataset,
                    vocabularies=self.vocabularies1,
                    bert_name=bert_name,
                )

                batch_size = len(batcher)
                with_tqdm = True

                tg.set_device('cuda:0') 
                device = tg.device

                pred_batcher = predict(batcher, self.classifier1, batch_size=64)
                
                scores = compute_scores(pred_batcher, batcher)
                
                cpt_batch += batch_len

#                 if any(len(tj) > 0 for tj in trajectory_scores.values()):
#                     trajectory_scores = [t for tj in trajectory_scores.values() for t in tj]
#                 else: policy_loss = 0

            self._optimizer.zero_grad()
            loss = (scores['loss'] * attns).mean() # policy_loss self._batch_loss(batch, for_training=True) + 
            loss.backward()

            # Make sure Variable is on the cpu before converting to numpy.
            # .cpu() is a no-op if you aren't using GPUs.
            train_loss += loss.data.cpu().numpy()
            batch_grad_norm = self._rescale_gradients()

            # This does nothing if batch_num_total is None or you are using an
            # LRScheduler which doesn't update per batch.
            if self._learning_rate_scheduler:
                self._learning_rate_scheduler.step_batch(batch_num_total)
                
            self._optimizer.step()

            # Update the description with the latest metrics
            metrics = self._get_metrics(train_loss, batches_this_epoch)
            description = self._description_from_metrics(metrics)

            train_generator_tqdm.set_description(description, refresh=False)
            if hasattr(self, "_tf_params") and self._tf_params is not None:
                # We have TF logging
                if self._batch_num_total % self._tf_params["log_every"] == 0:
                    self._tf_log(metrics, self._batch_num_total)

        return self._get_metrics(train_loss, batches_this_epoch, reset=True)
    
import functools

trainer._train_epoch = functools.partial(_train_epoch, trainer)

In [12]:
# ADD CUSTOM BERT IN allennlp/modules.token_embedders.bert_token_embedder.py -> BertEmbedder or PretrainedBertEmbedder

In [13]:
trainer.train()

2021-03-12 17:25:13,994: INFO: Beginning training.
2021-03-12 17:25:13,996: INFO: ==================================================
2021-03-12 17:25:13,996: INFO: Starting Training Epoch 1/50
2021-03-12 17:25:13,997: INFO: Peak CPU memory usage MB: 6201.82
2021-03-12 17:25:14,093: INFO: GPU 0 memory usage MB: 5746


/home/ytaille/AttentionSegmentation/allennlp/data/fields/array_field.py:42: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return_array[slices] = self.array
CompositeMention: 0.0525, DiseaseClass: 0.2445, Modifier: 0.4071, SpecificDisease: 0.6364, accuracy: 0.3351, loss: 0.0036 ||:  98%|█████████▊| 56/57 [01:18<00:01,  1.39s/it]


2021-03-12 17:26:33,880: ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-3435b262f1ae>", line 1, in <module>
    trainer.train()
  File "/home/ytaille/AttentionSegmentation/AttentionSegmentation/trainer.py", line 61, in train
    super(Trainer, self).train(*args, **kwargs)
  File "/home/ytaille/AttentionSegmentation/AttentionSegmentation/commons/trainer.py", line 588, in train
    train_metrics = self._train_epoch(epoch)
  File "<ipython-input-11-9003421f0f47>", line 176, in _train_epoch
    bert_name=bert_name,
  File "/home/ytaille/AttentionSegmentation/notebook_utils.py", line 488, in preprocess_train
    mentions["text"] = mentions["text"].str.lower()
KeyboardInterrupt

During handling of th

TypeError: object of type 'NoneType' has no len()

In [ ]:
logger.info("Training Done.")
if instances_test is not None:
    logger.info("Computing final Test Accuracy")
    trainer.test(instances_test)
logger.info("Done.")